In [4]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from graphviz import Digraph
import torch
import torch.nn.functional as F

# 1. Add regularization things, (W, batch norm.. etc)
# 2. Create Classes for layers
# 3. Create the MLP with 5 layers


In [5]:
words = open('names.txt', 'r').read().splitlines()

for word in words[:10]:
  print(word)

words = ["."] + [f"{word}." for word in words]
print(words)
chars = sorted(list(set(''.join(words))))
print(chars)

stoi = {s:i for i,s in enumerate(chars)}
print(stoi)
itos = {i:s for s,i in stoi.items()}
print(itos)

def build_dataset(words, block_size, is_print=False):  
  X, Y = [], []
  for word in words:
    context = [0] * block_size
    for ch in word:
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      if is_print:
        print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  # print(X.shape, Y.shape)
  return X, Y

emma
olivia
ava
isabella
sophia
charlotte
mia
amelia
harper
evelyn
['.', 'emma.', 'olivia.', 'ava.', 'isabella.', 'sophia.', 'charlotte.', 'mia.', 'amelia.', 'harper.', 'evelyn.', 'abigail.', 'emily.', 'elizabeth.', 'mila.', 'ella.', 'avery.', 'sofia.', 'camila.', 'aria.', 'scarlett.', 'victoria.', 'madison.', 'luna.', 'grace.', 'chloe.', 'penelope.', 'layla.', 'riley.', 'zoey.', 'nora.', 'lily.', 'eleanor.', 'hannah.', 'lillian.', 'addison.', 'aubrey.', 'ellie.', 'stella.', 'natalie.', 'zoe.', 'leah.', 'hazel.', 'violet.', 'aurora.', 'savannah.', 'audrey.', 'brooklyn.', 'bella.', 'claire.', 'skylar.', 'lucy.', 'paisley.', 'everly.', 'anna.', 'caroline.', 'nova.', 'genesis.', 'emilia.', 'kennedy.', 'samantha.', 'maya.', 'willow.', 'kinsley.', 'naomi.', 'aaliyah.', 'elena.', 'sarah.', 'ariana.', 'allison.', 'gabriella.', 'alice.', 'madelyn.', 'cora.', 'ruby.', 'eva.', 'serenity.', 'autumn.', 'adeline.', 'hailey.', 'gianna.', 'valentina.', 'isla.', 'eliana.', 'quinn.', 'nevaeh.', 'ivy.',

In [15]:
# params
g = torch.Generator().manual_seed(2147483647)

n_chars = 27
n_embd = 2
n_block_size = 3
n_hidden = 100
n_batch = 32
n_steps = 100000

# splits of training, dev and test
# 80%, 10%, 10%
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1], block_size=n_block_size)
Xval, Yval = build_dataset(words[n1:n2], block_size=n_block_size)
Xte, Yte = build_dataset(words[n2:], block_size=n_block_size)

In [9]:
# train
C = torch.randn((n_chars, n_embd), generator=g)
W1 = torch.randn((n_embd * n_block_size, n_hidden), generator=g) * (5/3)/((n_embd * n_block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.01
W2 = torch.randn((n_hidden, n_chars), generator=g) * 0.01
b2 = torch.randn(n_chars, generator=g) * 0

# BatchNorm parameters
bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))
bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
for parameter in parameters:
    parameter.requires_grad = True

for i in range(10000):

    # forward pass
    ix_batch = torch.randint(0, Xtr.shape[0], (n_batch,))
    emb = C[Xtr[ix_batch]]
    emb = emb.view(emb.shape[0], -1)
    # Linear layer
    hpreact = emb @ W1 + b1
    # BatchNorm layer
    bnmeani = hpreact.mean(0, keepdim=True)
    bnstdi = hpreact.std(0, keepdim=True)
    hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias
    with torch.no_grad():
        bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
        bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi
    # Non-linearity
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix_batch])

    # backward pass
    for parameter in parameters:
        parameter.grad = None
    loss.backward()

    # update
    for parameter in parameters:
        parameter.data -= 0.1 * parameter.grad

    print(f"{i=} {loss.item()=}")

# train loss
emb = C[Xtr]
emb = emb.view(emb.shape[0], -1)
# Linear layer
hpreact = emb @ W1 + b1
# BatchNorm layer
hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
# Non-linearity
h = torch.tanh(hpreact)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
print(f"train loss: {loss.item()=}")

# validation loss
emb = C[Xval]
emb = emb.view(emb.shape[0], -1)
# Linear layer
hpreact = emb @ W1 + b1
# BatchNorm layer
hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
# Non-linearity
h = torch.tanh(hpreact)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yval)
print(f"validation loss: {loss.item()=}")

i=0 loss.item()=3.294564962387085
i=1 loss.item()=3.273209810256958
i=2 loss.item()=3.2482879161834717
i=3 loss.item()=3.228785514831543
i=4 loss.item()=3.214296579360962
i=5 loss.item()=3.207674741744995
i=6 loss.item()=3.1721720695495605
i=7 loss.item()=3.1771609783172607
i=8 loss.item()=3.16391658782959
i=9 loss.item()=3.132821559906006
i=10 loss.item()=3.1359946727752686
i=11 loss.item()=3.1249616146087646
i=12 loss.item()=3.115108013153076
i=13 loss.item()=3.1031508445739746
i=14 loss.item()=3.0876340866088867
i=15 loss.item()=3.0838427543640137
i=16 loss.item()=3.084385871887207
i=17 loss.item()=3.0442843437194824
i=18 loss.item()=3.059872627258301
i=19 loss.item()=3.0233516693115234
i=20 loss.item()=3.05119252204895
i=21 loss.item()=3.008096933364868
i=22 loss.item()=3.015212297439575
i=23 loss.item()=2.9975314140319824
i=24 loss.item()=2.9760282039642334
i=25 loss.item()=2.9994657039642334
i=26 loss.item()=2.9938766956329346
i=27 loss.item()=2.996765613555908
i=28 loss.item()=2

In [29]:
class Linear:
  
  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out), generator=g) / fan_in**0.5
    self.bias = torch.zeros(fan_out) if bias else None
  
  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out
  
  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])
  
class BatchNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      xmean = x.mean(0, keepdim=True) # batch mean
      xvar = x.var(0, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]
  
class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []

In [39]:
# define
C = torch.randn((n_chars, n_embd), generator=g)
layers = [
    Linear(n_block_size*n_embd, n_hidden, bias=False),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden, n_hidden, bias=False),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden, n_hidden, bias=False),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden, n_hidden, bias=False),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden, n_hidden, bias=False),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden, n_chars, bias=False),
    BatchNorm1d(n_chars),
]

with torch.no_grad():
  # last layer: make less confident
  layers[-1].gamma *= 0.1
  # all other layers: apply gain
  for layer in layers[:-1]:
    if isinstance(layer, Linear):
      layer.weight *= 5/3

parameters = [C] + [parameter for layer in layers for parameter in layer.parameters()]
for parameter in parameters:
  parameter.requires_grad = True


In [43]:
# train
lossi = []
ud = []
for i in range(n_steps):

    # forward pass
    ix_batch = torch.randint(0, Xtr.shape[0], (n_batch,))
    emb = C[Xtr[ix_batch]]
    x = emb.view(emb.shape[0], -1)
    # All layers
    for layer in layers:
       x = layer(x)
    loss = F.cross_entropy(x, Ytr[ix_batch])

    # backward pass
    for parameter in parameters:
        parameter.grad = None
    loss.backward()

    # update
    lr = 0.1 if i < 150000 else 0.01 # step learning rate decay
    for parameter in parameters:
      parameter.data += -lr * parameter.grad

    # track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{n_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())
    with torch.no_grad():
      ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])

      0/ 100000: 3.2905
  10000/ 100000: 2.5008
  20000/ 100000: 2.2121
  30000/ 100000: 1.6823
  40000/ 100000: 2.7186
  50000/ 100000: 1.9792
  60000/ 100000: 2.1867
  70000/ 100000: 2.1241
  80000/ 100000: 2.2695
  90000/ 100000: 1.8837


In [44]:
# inference
g = torch.Generator().manual_seed(2147483647)

for layer in layers:
    if isinstance(layer, BatchNorm1d):
        layer.training = False
        
for i in range(10):
    context = [0] * n_block_size
    out = []
    while True:

        # Forward pass
        emb = C[torch.tensor([context])]
        x = emb.view(emb.shape[0], -1)
        # All layers
        for layer in layers:
            x = layer(x)
        p = F.softmax(x, dim=1)

        # Sample
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()

        # Update context
        context = context[1:] + [ix]

        # Append to output
        out.append(itos[ix])
        if ix == 0:
            break

    print(''.join(out))

dexz.
aleah.
mikion.
kayden.
maisiana.
nella.
kaman.
a.
samiyah.
javer.
